# COVID-19 Prediction Model

This notebook implements an end-to-end machine learning pipeline for COVID-19 hospitalization prediction:
- **Part 1**: Data cleaning and preprocessing
- **Part 2**: Exploratory data analysis
- **Part 3**: Model training and evaluation
- **Part 4**: Advanced validation and deployment

**Note**: See `documentation.md` for detailed methodology and design decisions.


# PART 1: DATA CLEANING & PREPROCESSING


## Load Dataset and Dependencies


Import the core libraries needed for data manipulation and numerical operations.

In [ ]:
import pandas as pd
import numpy as np

Load dataset with configurable sample size for faster processing:


The `SAMPLE_SIZE` variable controls how many rows to load. Using a smaller sample (like 400K instead of the full 700K) significantly reduces processing time while maintaining reasonable accuracy. Set to `None` to load the complete dataset.

In [ ]:
SAMPLE_SIZE = 400000  # Default: 400K rows for balanced speed/accuracy (~4-5 minutes)

if SAMPLE_SIZE is not None:
    df = pd.read_csv('../data/Covid_Data.csv', nrows=SAMPLE_SIZE)
    print(f"Loading sample: {SAMPLE_SIZE:,} rows")
else:
    df = pd.read_csv('../data/Covid_Data.csv')
    print("Loading full dataset...")

print(f"\nDataset shape: {df.shape}")
print(f"Rows: {df.shape[0]:,}, Columns: {df.shape[1]}")
df.head(20)

## Remove Data Leakage

Remove `DATE_DIED` - this is a post-outcome variable unavailable at prediction time:


**Why this matters:** If we kept DATE_DIED, the model would learn that "having a death date means severe COVID", but this information only exists AFTER the outcome occurs. In real-world prediction, we need to predict before knowing if someone will die.

In [ ]:
df.drop(columns=["DATE_DIED"], inplace=True)

print("After dropping DATE_DIED:", df.shape)

## Remove Demographic Bias

Remove `SEX` to focus on medical symptoms only:


**Why this matters:** While sex might correlate with COVID outcomes, including it could introduce bias in predictions. We want the model to focus purely on medical symptoms and conditions rather than demographic factors.

In [ ]:
df.drop(columns=["SEX"], inplace=True)

print("After dropping SEX:", df.shape)

## Encode Hospitalization Status

Rename and re-encode PATIENT_TYPE as HOSPITALIZED (1=hospitalized, 0=outpatient):


**What this does:** Converts the original encoding (1=outpatient, 2=hospitalized) to a more intuitive binary format (0=outpatient, 1=hospitalized) where 1 represents the more severe condition.

In [ ]:
# Rename column
df.rename(columns={"PATIENT_TYPE": "HOSPITALIZED"}, inplace=True)

# Re-encode values
df["HOSPITALIZED"] = df["HOSPITALIZED"].replace({1: 0, 2: 1})

print(df["HOSPITALIZED"].value_counts())

## Filter Valid Classifications

Keep only valid COVID classification codes (1-7):


**What this does:** Removes records with classification codes 97, 98, 99 which represent "unknown", "not applicable", or "missing" values. These would add noise to the model and reduce prediction accuracy.

In [ ]:
df = df[df["CLASIFFICATION_FINAL"].between(1, 7)]

print("After filtering CLASIFFICATION_FINAL:", df.shape)

## Create Binary Target Variable

Create `covid` target: classes 1-3 → 1 (positive), classes 4-7 → 0 (negative):


**What this does:** Converts the multi-class classification problem into binary classification. Classes 1-3 represent confirmed COVID cases, while 4-7 represent negative or non-COVID cases. This simplifies the prediction task.

In [ ]:
df["covid"] = df["CLASIFFICATION_FINAL"].apply(
    lambda x: 1 if x in [1, 2, 3] else 0
)

print(df["covid"].value_counts())

## Define Binary Features

List all binary medical condition features for encoding:


**What this does:** Creates a list of all medical condition columns that need binary encoding. These represent presence/absence of various health conditions and symptoms.

In [ ]:
binary_cols = [
    "USMER", "HOSPITALIZED", "INTUBED", "PNEUMONIA",
    "PREGNANT", "DIABETES", "COPD", "ASTHMA", "INMSUPR",
    "HIPERTENSION", "OTHER_DISEASE", "CARDIOVASCULAR",
    "OBESITY", "RENAL_CHRONIC", "TOBACCO", "ICU"
]

## Filter Binary Features

Keep only rows with valid binary values (1 or 2) for all medical features:


**What this does:** Removes rows containing missing or invalid values (97, 98, 99) in any medical condition column. This ensures data quality by keeping only complete, valid records.

In [ ]:
for col in binary_cols:
    df = df[df[col].isin([1, 2])]

print("After strict binary filtering:", df.shape)

## Standardize Binary Encoding

Convert all binary features to 0/1 format (1→1, 2→0):


**What this does:** Transforms the original encoding (1=yes, 2=no) to standard binary format (1=yes, 0=no). Machine learning models work better with 0/1 encoding as it's mathematically cleaner.

In [ ]:
df[binary_cols] = df[binary_cols].replace({1: 1, 2: 0})

print("Binary encoding completed (1 → 1, 2 → 0).")

## Clean Age Column

Convert AGE to numeric and filter valid range (0-120):


**What this does:** Ensures age values are numeric and biologically plausible. Any non-numeric values are converted to NaN (coercion), and ages outside 0-120 range are filtered out as they're likely data entry errors.

In [ ]:
df["AGE"] = pd.to_numeric(df["AGE"], errors="coerce")
df = df[(df["AGE"] >= 0) & (df["AGE"] <= 120)]

print("After AGE cleaning:", df.shape)

## Remove Classification Column

Drop CLASIFFICATION_FINAL after creating target variable:


In [ ]:
df.drop(columns=["CLASIFFICATION_FINAL"], inplace=True)

print("Final cleaned dataset shape:", df.shape)

## Validate Data Quality

Verify all binary columns contain only 0 or 1:


In [ ]:
# Ensure all binary columns contain only 0 or 1
for col in binary_cols:
    assert set(df[col].unique()).issubset({0, 1})

print("All binary columns are clean (0/1 only)")
df.head()

# PART 2: EXPLORATORY DATA ANALYSIS


## Import ML Libraries


In [ ]:
from sklearn.model_selection import train_test_split, cross_val_score, learning_curve, StratifiedKFold, cross_validate
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    confusion_matrix, 
    roc_curve,
    auc,
)

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier

import joblib
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Set visualization style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print("Libraries imported.")

## Visualize Target Distribution

Plot COVID-19 case distribution (count and percentage):


**What this shows:** The balance between COVID positive and negative cases in our dataset. Understanding class distribution helps identify if we have imbalanced data, which could affect model performance.

In [ ]:
# Target distribution
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Count plot
covid_counts = df["covid"].value_counts()
axes[0].bar(['COVID Negative', 'COVID Positive'], covid_counts.values, 
            color=['lightgreen', 'lightcoral'], edgecolor='black')
axes[0].set_title('COVID-19 Distribution (Count)', fontweight='bold', fontsize=14)
axes[0].set_ylabel('Number of Patients')
axes[0].grid(axis='y', alpha=0.3)

# Add counts on bars
for i, v in enumerate(covid_counts.values):
    axes[0].text(i, v + 100, str(v), ha='center', fontweight='bold')

# Pie chart
axes[1].pie(covid_counts.values, labels=['COVID Negative', 'COVID Positive'], 
            autopct='%1.1f%%', colors=['lightgreen', 'lightcoral'], 
            startangle=90, textprops={'fontsize': 12, 'fontweight': 'bold'})
axes[1].set_title('COVID-19 Distribution (%)', fontweight='bold', fontsize=14)

plt.tight_layout()
plt.show()

print(f"\nCOVID Distribution:")
print(f"Negative: {covid_counts[0]:,} ({covid_counts[0]/len(df)*100:.2f}%)")
print(f"Positive: {covid_counts[1]:,} ({covid_counts[1]/len(df)*100:.2f}%)")
print(f"Total: {len(df):,}")

## Correlation Analysis

Generate correlation matrix and identify top features correlated with COVID:


**What this shows:** How strongly each medical feature correlates with COVID status. Positive correlations (closer to +1) mean the feature increases with COVID presence, negative correlations (closer to -1) mean it decreases. Features with strong correlations are likely important predictors.

In [ ]:
# Correlation matrix
correlation_matrix = df.corr()

plt.figure(figsize=(16, 12))
sns.heatmap(correlation_matrix, annot=True, fmt='.2f', cmap='coolwarm', 
            square=True, linewidths=0.5, cbar_kws={"shrink": 0.8}, 
            center=0, vmin=-1, vmax=1)
plt.title('Correlation Matrix - All Medical Features', fontweight='bold', fontsize=16)
plt.tight_layout()
plt.show()

# Correlation with target
print(f"\nTop features correlated with COVID:")
target_corr = df.corr()["covid"].sort_values(ascending=False)
for feat, corr in target_corr.items():
    if feat != "covid":
        print(f"{feat:20s}: {corr:+.4f}")

## Feature Distributions

Visualize distribution of all binary medical conditions:


**What this shows:** The frequency of each medical condition in the dataset. This helps identify common vs rare conditions and ensures we have sufficient data for each feature to make meaningful predictions.

## Age Distribution Analysis

Compare age distributions between COVID positive and negative cases:


**What this shows:** How age distribution differs between COVID positive and negative patients. This visualization reveals if certain age groups are more susceptible to COVID, which is valuable medical insight.

In [ ]:
# Binary features distribution
binary_features = [col for col in df.columns if col != 'covid' and col != 'AGE' and col not in ['USMER', 'MEDICAL_UNIT']]

n_features = len(binary_features)
n_cols = 4
n_rows = (n_features + n_cols - 1) // n_cols

fig, axes = plt.subplots(n_rows, n_cols, figsize=(16, n_rows * 3))
axes = axes.flatten()

for idx, feature in enumerate(binary_features):
    counts = df[feature].value_counts().sort_index()
    axes[idx].bar(['No', 'Yes'], counts.values if len(counts) == 2 else [counts.get(0, 0), counts.get(1, 0)],
                  color=['lightblue', 'salmon'], edgecolor='black')
    axes[idx].set_title(f'{feature}', fontweight='bold')
    axes[idx].set_ylabel('Count')
    axes[idx].grid(axis='y', alpha=0.3)

# Hide extra subplots
for idx in range(n_features, len(axes)):
    axes[idx].axis('off')

plt.suptitle('Distribution of Medical Conditions', fontsize=16, fontweight='bold', y=1.00)
plt.tight_layout()
plt.show()

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Age distribution by COVID status
for status in [0, 1]:
    label = 'COVID Negative' if status == 0 else 'COVID Positive'
    color = 'lightgreen' if status == 0 else 'lightcoral'
    axes[0].hist(df[df['covid'] == status]['AGE'], bins=30, alpha=0.7, 
                 label=label, color=color, edgecolor='black')

axes[0].set_xlabel('Age', fontweight='bold')
axes[0].set_ylabel('Frequency', fontweight='bold')
axes[0].set_title('Age Distribution by COVID Status', fontweight='bold', fontsize=14)
axes[0].legend()
axes[0].grid(axis='y', alpha=0.3)

# Box plot
age_data = [df[df['covid'] == 0]['AGE'], df[df['covid'] == 1]['AGE']]
bp = axes[1].boxplot(age_data, labels=['COVID Negative', 'COVID Positive'], 
                      patch_artist=True)
for patch, color in zip(bp['boxes'], ['lightgreen', 'lightcoral']):
    patch.set_facecolor(color)
axes[1].set_ylabel('Age', fontweight='bold')
axes[1].set_title('Age Distribution (Box Plot)', fontweight='bold', fontsize=14)
axes[1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

# Age statistics
print(f"\nAge Statistics by COVID Status:")
for status in [0, 1]:
    label = 'Negative' if status == 0 else 'Positive'
    age_data = df[df['covid'] == status]['AGE']
    print(f"\n{label}: Mean={age_data.mean():.1f}, Median={age_data.median():.1f}, Std={age_data.std():.1f}")

# PART 3: MODEL TRAINING & EVALUATION


## Prepare Features and Target


In [ ]:
X = df.drop(columns=["covid"])
y = df["covid"]

print(f"Features shape: {X.shape}")
print(f"Target shape: {y.shape}")

## Split Data (Stratified)

80/20 train-test split maintaining class distribution:


**What this does:** Divides data into 80% for training models and 20% for testing. Stratification ensures both sets have the same proportion of COVID positive/negative cases, preventing bias in evaluation.

In [ ]:
# Use stratified split to maintain class distribution in train/test sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Training set: {X_train.shape}")
print(f"Test set: {X_test.shape}")
print(f"\nClass distribution in training set:")
print(y_train.value_counts(normalize=True))
print(f"\nClass distribution in test set:")
print(y_test.value_counts(normalize=True))

In [ ]:
from sklearn.preprocessing import StandardScaler

# Apply StandardScaler (0 mean, 1 std deviation)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("Features scaled using StandardScaler")
print(f"AGE range: [{X_train['AGE'].min():.0f}, {X_train['AGE'].max():.0f}] -> [{X_train_scaled[:, X_train.columns.get_loc('AGE')].min():.2f}, {X_train_scaled[:, X_train.columns.get_loc('AGE')].max():.2f}]")

## Feature Scaling

Apply StandardScaler for distance-based algorithms (improves KNN and LogReg):


**What this does:** Transforms features to have mean=0 and standard deviation=1. This is crucial for distance-based algorithms (KNN, Logistic Regression) because features with larger ranges would otherwise dominate the calculations. For example, AGE (0-120) would overpower binary features (0-1) without scaling.

## Define Evaluation Function


**What this does:** Creates a reusable function to calculate key performance metrics: Accuracy (% correct), Precision (% of positive predictions that are correct), Recall (% of actual positives detected), and F1-Score (harmonic mean of precision and recall).

In [ ]:
def evaluate_model(name, y_true, y_pred):
    """Evaluate model performance"""
    acc = accuracy_score(y_true, y_pred)
    prec = precision_score(y_true, y_pred)
    rec = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    
    print(f"\n{name}")
    print(f"Accuracy: {acc:.4f}, Precision: {prec:.4f}, Recall: {rec:.4f}, F1: {f1:.4f}")

## Train Model 1: Logistic Regression


**What this does:** Trains a linear classification model that predicts probabilities using a logistic function. It's fast, interpretable, and works well as a baseline. Uses scaled data because it's sensitive to feature magnitudes.

In [ ]:
log_reg = LogisticRegression(max_iter=1000, random_state=42)
log_reg.fit(X_train_scaled, y_train)  # Use scaled data
y_pred_lr = log_reg.predict(X_test_scaled)  # Use scaled test data

print("Logistic Regression trained with scaled features")

## Train Model 2: Decision Tree (Regularized)

Regularization prevents overfitting with max_depth, min_samples constraints:


**What this does:** Trains a tree-based model that makes decisions using if-then rules. Regularization parameters (max_depth, min_samples) prevent the tree from becoming too complex and memorizing training data. Class weights handle imbalanced data.

In [ ]:
# Add regularization to prevent overfitting
dt = DecisionTreeClassifier(
    max_depth=8,               # Limit tree depth
    min_samples_split=20,      # Require more samples to split
    min_samples_leaf=10,       # Require more samples in leaf nodes
    random_state=42,
    class_weight='balanced'    # Handle class imbalance
)
dt.fit(X_train, y_train)
y_pred_dt = dt.predict(X_test)

print("Decision Tree trained with regularization")

## Train Model 3: Random Forest (Regularized)

Reduced trees and added constraints to improve generalization:


**What this does:** Trains an ensemble of 100 decision trees, each seeing different subsets of data. Predictions are made by majority vote. This reduces overfitting compared to a single tree and often achieves better accuracy.

In [ ]:
# Add regularization to prevent overfitting
rf = RandomForestClassifier(
    n_estimators=100,          # Reduced from 300 to prevent overfitting
    max_depth=10,              # Limit tree depth
    min_samples_split=20,      # Require more samples to split
    min_samples_leaf=10,       # Require more samples in leaf nodes
    max_features='sqrt',       # Use sqrt of features at each split
    random_state=42,
    n_jobs=-1,
    class_weight='balanced'    # Handle class imbalance
)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)

print("Random Forest trained with regularization")

## Train Model 4: K-Nearest Neighbors


**What this does:** Predicts by finding the 5 most similar training samples and using their majority class. Requires scaled data because it uses distance calculations. Simple but can be effective for well-clustered data.

In [ ]:
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train_scaled, y_train)  # Use scaled data
y_pred_knn = knn.predict(X_test_scaled)  # Use scaled test data

print("KNN trained with scaled features")

## Train Model 5: Gradient Boosting (Regularized)


**What this does:** Builds trees sequentially where each new tree corrects errors from previous trees. Often achieves highest accuracy but requires careful tuning to prevent overfitting. Regularization parameters control complexity.

In [ ]:
# Add regularization to prevent overfitting
gb = GradientBoostingClassifier(
    n_estimators=100,
    max_depth=5,               # Limit tree depth
    min_samples_split=20,      # Require more samples to split
    min_samples_leaf=10,       # Require more samples in leaf nodes
    learning_rate=0.1,         # Slower learning for better generalization
    subsample=0.8,             # Use 80% of samples for each tree
    random_state=42
)
gb.fit(X_train, y_train)
y_pred_gb = gb.predict(X_test)

print("Gradient Boosting trained with regularization")

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

# Define parameter distributions (minimal for speed)
param_distributions = {
    'n_estimators': [80, 100, 120],      # Fewer trees = faster
    'max_depth': [4, 5, 6],
    'min_samples_split': [15, 20],
    'min_samples_leaf': [8, 10],
    'learning_rate': [0.08, 0.1, 0.12],
    'subsample': [0.8]                   # Single value = no search
}

print("Starting hyperparameter tuning (8 combinations, 2-fold CV)...")

# ULTRA-FAST configuration
random_search = RandomizedSearchCV(
    GradientBoostingClassifier(random_state=42),
    param_distributions,
    n_iter=8,              # Only 8 combinations!
    cv=2,                  # 2-fold CV = 16 model fits total
    scoring='accuracy',
    n_jobs=-1,
    verbose=0,             # No verbose output for speed
    random_state=42
)

# Fit and find best parameters
random_search.fit(X_train, y_train)

print(f"{'='*70}")
print("BEST PARAMETERS FOUND:")
print(f"\nBest parameters:")
for param, value in random_search.best_params_.items():
    print(f"  {param}: {value}")
print(f"Best CV Accuracy: {random_search.best_score_:.4f}")

# Use the best model
gb_tuned = random_search.best_estimator_
y_pred_gb_tuned = gb_tuned.predict(X_test)

print("Optimized model trained.")

In [ ]:
from sklearn.ensemble import VotingClassifier

# Create voting ensemble combining top 3 models
voting_clf = VotingClassifier(
    estimators=[
        ('gb_tuned', gb_tuned),      # Optimized Gradient Boosting (weight=2)
        ('rf', rf),                   # Random Forest (weight=1)
        ('log_reg', log_reg)          # Logistic Regression (weight=1)
    ],
    voting='soft',                    # Use probability voting
    weights=[2, 1, 1]                 # GB gets double weight (best performer)
)

print("Training ensemble model...")
voting_clf.fit(X_train_scaled, y_train)
y_pred_voting = voting_clf.predict(X_test_scaled)

print("Ensemble trained (GB=50%, RF=25%, LR=25%)")

## Train Model 7: Voting Ensemble

Combines Gradient Boosting (50%), Random Forest (25%), and Logistic Regression (25%):


**What this does:** Combines predictions from three models using weighted voting. Gradient Boosting gets double weight because it's typically the best performer. This ensemble approach often outperforms individual models by leveraging their different strengths.

## Train Model 6: Gradient Boosting (Hyperparameter Tuned)

Fast RandomizedSearchCV with 8 combinations × 2-fold CV = 16 fits (~10-20 seconds):


**What this does:** Automatically searches for the best hyperparameters by testing 8 random combinations from defined ranges. Uses 2-fold cross-validation for speed while still getting reliable estimates. This often improves accuracy by 2-3%.

## Evaluate All Models

Compare performance across all 7 trained models:


In [ ]:
evaluate_model("Logistic Regression (Scaled)", y_test, y_pred_lr)
evaluate_model("Decision Tree", y_test, y_pred_dt)
evaluate_model("Random Forest", y_test, y_pred_rf)
evaluate_model("KNN (Scaled)", y_test, y_pred_knn)
evaluate_model("Gradient Boosting", y_test, y_pred_gb)
evaluate_model("Gradient Boosting (Tuned)", y_test, y_pred_gb_tuned)
evaluate_model("Voting Ensemble", y_test, y_pred_voting)

## Compare Model Performance

Generate comparison table and visualizations:


**What this does:** Evaluates all 7 models using both test accuracy and 5-fold cross-validation. Cross-validation provides more reliable estimates by testing on multiple data splits. The visualizations make it easy to identify the best performing model.

In [ ]:
# Store all models and predictions
models = {
    'Logistic Regression': (log_reg, y_pred_lr, X_train_scaled),
    'Decision Tree': (dt, y_pred_dt, X_train),
    'Random Forest': (rf, y_pred_rf, X_train),
    'KNN': (knn, y_pred_knn, X_train_scaled),
    'Gradient Boosting': (gb, y_pred_gb, X_train),
    'GB (Tuned)': (gb_tuned, y_pred_gb_tuned, X_train),
    'Voting Ensemble': (voting_clf, y_pred_voting, X_train_scaled)
}

# Evaluate all models
results = []
for name, (model, y_pred, X_train_data) in models.items():
    test_acc = accuracy_score(y_test, y_pred)
    cv_scores = cross_val_score(model, X_train_data, y_train, cv=5, n_jobs=-1)
    
    results.append({
        'Model': name,
        'Test Accuracy': test_acc,
        'CV Mean': cv_scores.mean(),
        'CV Std': cv_scores.std()
    })

results_df = pd.DataFrame(results).sort_values('Test Accuracy', ascending=False)

print(f"\nModel Comparison:")
print(results_df.to_string(index=False))
print(f"\nBest: {results_df.iloc[0]['Model']} ({results_df.iloc[0]['Test Accuracy']:.4f})\n")

# Visualization
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Test accuracy
axes[0].barh(results_df['Model'], results_df['Test Accuracy'])
axes[0].set_xlabel('Accuracy')
axes[0].set_title('Test Accuracy Comparison', fontweight='bold')
axes[0].set_xlim([0.5, 1.0])
axes[0].grid(axis='x', alpha=0.3)

# Cross-validation accuracy
axes[1].barh(results_df['Model'], results_df['CV Mean'], color='coral')
axes[1].set_xlabel('Accuracy')
axes[1].set_title('Cross-Validation Accuracy', fontweight='bold')
axes[1].set_xlim([0.5, 1.0])
axes[1].grid(axis='x', alpha=0.3)

plt.tight_layout()
plt.show()

# Set best model
best_model_name = results_df.iloc[0]['Model']
if 'Voting' in best_model_name:
    best_model = voting_clf
elif 'Tuned' in best_model_name:
    best_model = gb_tuned
elif 'Gradient' in best_model_name:
    best_model = gb
elif 'Random' in best_model_name:
    best_model = rf
else:
    best_model = gb_tuned  # Default to tuned GB

print(f"\nBest model set to: {best_model_name}")

## Feature Importance Analysis

Analyze which features are most important for prediction:


**What this shows:** Which medical features the model relies on most heavily for predictions. High importance features are the key indicators the model uses to distinguish COVID positive from negative cases. This provides medical insights into disease indicators.

In [ ]:
# Feature importance from best model (Gradient Boosting)
feature_importance = pd.DataFrame({
    'Feature': X.columns,
    'Importance': best_model.feature_importances_
}).sort_values('Importance', ascending=False)

print(f"\nFeature Importance:")
print(feature_importance.to_string(index=False))

# Visualize top 15 features
plt.figure(figsize=(12, 8))
top_features = feature_importance.head(15)
plt.barh(top_features['Feature'], top_features['Importance'], 
         color='forestgreen', edgecolor='black')
plt.xlabel('Importance Score', fontweight='bold', fontsize=12)
plt.title('Top 15 Most Important Features for COVID-19 Prediction', 
          fontweight='bold', fontsize=14)
plt.gca().invert_yaxis()
plt.grid(axis='x', alpha=0.3)
plt.tight_layout()
plt.show()

# PART 4: ADVANCED VALIDATION & DEPLOYMENT


## Learning Curves Analysis

Visualize training vs validation performance to detect overfitting:


**What this shows:** How model performance changes with more training data. A large gap between training and validation curves indicates overfitting (model memorizing training data). Small gap indicates good generalization.

In [ ]:
print("\nCalculating learning curves...")
# Calculate learning curves
train_sizes, train_scores, val_scores = learning_curve(
    best_model,
    X_train,
    y_train,
    cv=5,
    n_jobs=-1,
    train_sizes=np.linspace(0.1, 1.0, 10),
    scoring='accuracy'
)

train_mean = np.mean(train_scores, axis=1)
train_std = np.std(train_scores, axis=1)
val_mean = np.mean(val_scores, axis=1)
val_std = np.std(val_scores, axis=1)

# Plot learning curves
plt.figure(figsize=(12, 7))
plt.plot(train_sizes, train_mean, label='Training Accuracy', 
         color='blue', marker='o', linewidth=2)
plt.fill_between(train_sizes, train_mean - train_std, train_mean + train_std, 
                 alpha=0.15, color='blue')

plt.plot(train_sizes, val_mean, label='Validation Accuracy (Cross-Val)', 
         color='red', marker='s', linewidth=2)
plt.fill_between(train_sizes, val_mean - val_std, val_mean + val_std, 
                 alpha=0.15, color='red')

plt.xlabel('Training Set Size', fontweight='bold', fontsize=12)
plt.ylabel('Accuracy', fontweight='bold', fontsize=12)
plt.title('Learning Curves - Best Model\n(Training vs Validation Performance)', fontweight='bold', fontsize=14)
plt.legend(loc='best', fontsize=11)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

# Statistics
overfitting_gap = train_mean[-1] - val_mean[-1]
print(f"\nTrain: {train_mean[-1]:.4f} ± {train_std[-1]:.4f}")
print(f"Valid: {val_mean[-1]:.4f} ± {val_std[-1]:.4f}")
print(f"Gap: {overfitting_gap:.4f}", end="")

if overfitting_gap < 0.05:
    print(" (good)")
elif overfitting_gap < 0.10:
    print(" (moderate overfitting)")
else:
    print(" (high overfitting)")

## 10-Fold Cross-Validation

Comprehensive validation using stratified 10-fold CV:


**What this does:** Splits data into 10 parts, trains on 9 and tests on 1, rotating through all combinations. This provides more robust performance estimates than a single train-test split by testing on multiple data combinations.

In [ ]:
print("\n10-Fold Cross-Validation")

best_model = gb

skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
cv_results = cross_validate(
    best_model,
    X_train,
    y_train,
    cv=skf,
    scoring=['accuracy', 'precision', 'recall', 'f1'],
    return_train_score=True
)

print("Cross-Validation Results (10 Folds):")
print(f"{'='*80}")
print(f"Training Accuracy:   {cv_results['train_accuracy'].mean():.4f} ± {cv_results['train_accuracy'].std():.4f}")
print(f"\nResults:")
print(f"Train Acc:  {cv_results['train_accuracy'].mean():.4f} ± {cv_results['train_accuracy'].std():.4f}")
print(f"Valid Acc:  {cv_results['test_accuracy'].mean():.4f} ± {cv_results['test_accuracy'].std():.4f}")
print(f"Precision:  {cv_results['test_precision'].mean():.4f} ± {cv_results['test_precision'].std():.4f}")
print(f"Recall:     {cv_results['test_recall'].mean():.4f} ± {cv_results['test_recall'].std():.4f}")
print(f"F1:         {cv_results['test_f1'].mean():.4f} ± {cv_results['test_f1'].std():.4f}")

overfitting_gap = cv_results['train_accuracy'].mean() - cv_results['test_accuracy'].mean()
print(f"Gap: {overfitting_gap:.4f}")

## ROC Curve & AUC Score

Evaluate model's discrimination ability:


**What this shows:** The model's ability to distinguish between positive and negative cases across all possible thresholds. AUC (Area Under Curve) of 1.0 is perfect, 0.5 is random guessing. Higher AUC means better discrimination ability.

In [ ]:
print("\nROC Analysis")

# Get probability predictions
y_pred_proba = best_model.predict_proba(X_test)[:, 1]

# Calculate ROC curve
fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)
roc_auc = auc(fpr, tpr)

# Plot ROC curve
plt.figure(figsize=(10, 8))
plt.plot(fpr, tpr, color='darkorange', lw=3, 
         label=f'ROC Curve (AUC = {roc_auc:.4f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', 
         label='Random Classifier (AUC = 0.50)')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate (1 - Specificity)', fontweight='bold', fontsize=12)
plt.ylabel('True Positive Rate (Sensitivity/Recall)', fontweight='bold', fontsize=12)
plt.title('ROC Curve - COVID-19 Prediction Model', fontweight='bold', fontsize=14)
plt.legend(loc="lower right", fontsize=12)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

print(f"ROC-AUC Score: {roc_auc:.4f}\n")
print("Interpretation:")
print(f"\nAUC: {roc_auc:.4f}")

## Confusion Matrix Analysis

Detailed breakdown of prediction errors (FN are critical in medical diagnosis):


**What this shows:** Detailed breakdown of correct and incorrect predictions. In medical diagnosis, False Negatives (missing COVID cases) are most critical as they mean infected patients go untreated. True Positives and True Negatives show correct predictions.

In [ ]:
# Enhanced confusion matrix
y_pred_final = best_model.predict(X_test)
cm = confusion_matrix(y_test, y_pred_final)

plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='RdYlGn', cbar=True,
            xticklabels=['COVID Negative (0)', 'COVID Positive (1)'],
            yticklabels=['COVID Negative (0)', 'COVID Positive (1)'],
            annot_kws={'size': 18, 'weight': 'bold'})
plt.title('Confusion Matrix - Final Model', fontweight='bold', fontsize=16)
plt.ylabel('Actual Label', fontweight='bold', fontsize=12)
plt.xlabel('Predicted Label', fontweight='bold', fontsize=12)
plt.tight_layout()
plt.show()

# Calculate metrics
tn, fp, fn, tp = cm.ravel()
specificity = tn / (tn + fp)
sensitivity = tp / (tp + fn)

print(f"\nResults:")
print(f"TN: {tn:,}  FP: {fp:,}")
print(f"FN: {fn:,}  TP: {tp:,}")
print(f"\nSensitivity: {sensitivity:.4f}")
print(f"Specificity: {specificity:.4f}")

## Final Performance Summary

Comprehensive summary of all validation metrics:


**What this shows:** A complete performance report combining all evaluation metrics from previous analyses. This provides a comprehensive view of model quality, reliability, and generalization capability.

In [ ]:
final_accuracy = accuracy_score(y_test, y_pred_final)
final_precision = precision_score(y_test, y_pred_final)
final_recall = recall_score(y_test, y_pred_final)
final_f1 = f1_score(y_test, y_pred_final)

print(f"\nFinal Performance Summary\n")

summary_data = {
    'Metric': [
        'Test Accuracy',
        'Cross-Val Accuracy (10-fold)',
        'Precision',
        'Recall (Sensitivity)',
        'F1-Score',
        'Specificity',
        'ROC-AUC Score',
        'Overfitting Gap'
    ],
    'Score': [
        f"{final_accuracy:.4f}",
        f"{cv_results['test_accuracy'].mean():.4f} ± {cv_results['test_accuracy'].std():.4f}",
        f"{final_precision:.4f}",
        f"{final_recall:.4f}",
        f"{final_f1:.4f}",
        f"{specificity:.4f}",
        f"{roc_auc:.4f}",
        f"{overfitting_gap:.4f}"
    ],
    'Interpretation': [
        f"{final_accuracy*100:.2f}% correct predictions",
        "Consistent across 10 folds",
        "% of positive predictions correct",
        "% of actual COVID cases detected",
        "Balance between precision/recall",
        "% of negatives correctly identified",
        "Overall discrimination ability",
        "Generalization quality"
    ]
}

summary_df = pd.DataFrame(summary_data)
print(summary_df.to_string(index=False))

print(f"\n{'='*80}")

## Save Model for Deployment

Save best model as .pkl file for production use:


**What this does:** Serializes the trained model to a file that can be loaded later for making predictions on new data. This enables deployment in production applications, APIs, or web services without retraining.

In [ ]:
# Save model
MODEL_DIR = "../model"
os.makedirs(MODEL_DIR, exist_ok=True)

model_path = os.path.join(MODEL_DIR, "covid_model.pkl")
joblib.dump(best_model, model_path)

print(f"Model saved: {model_path}")